# Lab 6 - Convolutional Neural Networks
by Rebecca Kuhlman, Sam Yao, and Michael Amberg



## Preparation

[1.5 points] 
Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

Because we will be dealing with identifying brain tumors, we want to use Recall. The equation for Recall takes into account False Negatives, which would be very bad if you falsely cleared someone of brain tumors, but they did in fact have a tumor.

[1.5 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice.

We could try Stratified 10 k-fold validation, because it seemed to be effective from the results we had in the last lab. I have no idea if this would extend to image data, but we could give it a try.

## Modeling

[1.5 points]  Setup the training to use data expansion in Keras (also called data augmentation). Explain why the chosen data expansion techniques are appropriate for your dataset. You can use the keras ImageGenerator as a pre-processing step OR in the optimization loop. You can also use the Keras-cv augmenter (a separate package: https://keras.io/keras_cv/ Links to an external site.)

In [ ]:
import 